In [1]:

import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

locals().update(license_keys)

os.environ.update(license_keys)

Saving Medical Language Models for Data Scientists  Training License.json to Medical Language Models for Data Scientists  Training License.json


In [24]:

import json, os




license1_fname="/content/Medical Language Models for Data Scientists  Training License (1).json"




with open(license1_fname) as f:

   license_keys = json.load(f)




# Adding license key-value pairs to environment variables

os.environ.update(license_keys)




# Defining license key-value pairs as local variables


In [2]:
!pip install -q pyspark spark-nlp spark-nlp-display

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.8/620.8 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.0 MB/s eta 0:00:00


In [3]:
!pip install -q johnsnowlabs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.1/268.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.3/683.3 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.2/579.2 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.0 MB/s eta

In [9]:
import sparknlp
from sparknlp.base import *
from sparknlp import *
from sparknlp.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

from johnsnowlabs import nlp, medical
nlp.install()

spark = nlp.start()

👌 Detected license file /content/Medical Language Models for Data Scientists  Training License.json
👌 Detected license file /content/Medical Language Models for Data Scientists  Training License.json
👌 JSL-Home is up to date! 
👌 Everything is already installed, no changes made
Spark Session already created, some configs may not take.
👌 Detected license file /content/Medical Language Models for Data Scientists  Training License.json


In [11]:
import sparknlp_jsl
from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *


In [12]:
def custom_pipeline_model():
  documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

  sentencer = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare", "en", "clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

  tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

  word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("word_embeddings")

  # adverse drug events
  ade_classifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_ade_augmented", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("ade_classification")

  ade_classifier1 = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_ade_augmented", "en", "clinical/models")\
    .setInputCols(["sentence", "token", "word_embeddings"])\
    .setOutputCol("ade_classification1")

  # to get NER
  clinical_ner = MedicalNerModel().pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token", "word_embeddings"])\
    .setOutputCol("clinical_ner")

  clinical_ner_chunk = NerConverterInternal()\
    .setInputCols(["sentence", "token", "clinical_ner"])\
    .setOutputCol("clinical_ner_chunk")

  # assertion model
  assertion = AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models")\
    .setInputCols(["sentence", "clinical_ner_chunk", "word_embeddings"])\
    .setOutputCol("assertion")\
    .setEntityAssertionCaseSensitive(False)

  # to get drug dosages
  posology_ner = MedicalNerModel().pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols(["sentence", "token", "word_embeddings"])\
    .setOutputCol("posology_ner")

  posology_ner_chunk = NerConverterInternal()\
    .setInputCols(["sentence", "token", "posology_ner"])\
    .setOutputCol("posology_ner_chunk")\
    .setWhiteList(["DRUG", "DOSAGE", "DURATION"])

  # ner deidentification
  deid_ner = MedicalNerModel().pretrained("ner_deid_generic_augmented", "en", "clinical/models")\
    .setInputCols(["sentence", "token", "word_embeddings"])\
    .setOutputCol("deid_ner")

  deid_ner_chunk = NerConverterInternal()\
    .setInputCols(["sentence", "token", "deid_ner"])\
    .setOutputCol("deid_ner_chunk")

  chunk_merger = ChunkMergeApproach()\
    .setInputCols(["clinical_ner_chunk", "ade_classification1", "assertion", "posology_ner_chunk", "deid_ner_chunk"])\
    .setOutputCol("chunk_merge")

  obfuscation = DeIdentification()\
    .setInputCols(["sentence","token","deid_ner_chunk"])\
    .setOutputCol("obfuscation")\
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefSource("faker")\
    .setMetadataMaskingPolicy("entity_labels")\
    .setOutputAsDocument(True)

  assertion_vop = AssertionDLModel.pretrained("assertion_vop_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "chunk_merge", "word_embeddings"])\
    .setOutputCol("assertion_vop")

  pos_tagger = PerceptronModel().pretrained("pos_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")

  dependency_parser = DependencyParserModel().pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "token", "pos_tags"])\
    .setOutputCol("dependency")

  generic_re = RelationExtractionModel().pretrained("generic_re")\
    .setInputCols(["word_embeddings", "pos_tags", "posology_ner_chunk", "dependency"])\
    .setOutputCol("generic_re")\
    .setMaxSyntacticDistance(10)

  # convert chunks to doc to get sentence embeddings of them
  chunk2doc = Chunk2Doc()\
    .setInputCols(["chunk_merge"])\
    .setOutputCol("chunk_doc")

  sbiobert_embedder = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["chunk_doc"])\
    .setOutputCol("sbiobert_embeddings")\
    .setCaseSensitive(False)

  # filter PROBLEM entity embeddings
  router_sentence_icd10 = Router()\
    .setInputCols(["sbiobert_embeddings"])\
    .setFilterFieldsElements(["PROBLEM"])\
    .setOutputCol("problem_embeddings")

  icd_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_slim_billable_hcc", "en", "clinical/models")\
    .setInputCols(["problem_embeddings"])\
    .setOutputCol("icd10cm_code")\
    .setDistanceFunction("EUCLIDEAN")

  # filter DRUG entity embeddings
  router_sentence_rxnorm = Router()\
    .setInputCols(["sbiobert_embeddings"])\
    .setFilterFieldsElements(["DRUG"])\
    .setOutputCol("drug_embeddings")

  rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented", "en", "clinical/models")\
    .setInputCols(["drug_embeddings"])\
    .setOutputCol("rxnorm_code")\
    .setDistanceFunction("EUCLIDEAN")

  summarizer = MedicalSummarizer.pretrained("summarizer_clinical_jsl")\
    .setInputCols(["document"])\
    .setOutputCol("summary")\
    .setMaxTextLength(512)\
    .setMaxNewTokens(512)

  pipeline = Pipeline(stages=[
      documenter,
      sentencer,
      tokenizer,
      word_embeddings,
      ade_classifier,
      ade_classifier1,
      clinical_ner,
      clinical_ner_chunk,
      assertion,
      posology_ner,
      posology_ner_chunk,
      deid_ner,
      deid_ner_chunk,
      chunk_merger,
      obfuscation,
      assertion_vop,
      pos_tagger,
      dependency_parser,
      generic_re,
      chunk2doc,
      sbiobert_embedder,
      router_sentence_icd10,
      icd_resolver,
      router_sentence_rxnorm,
      rxnorm_resolver,
      summarizer
  ])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  pipeline_model = pipeline.fit(empty_data)

  return pipeline_model


custom_model = custom_pipeline_model()

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
bert_sequence_classifier_ade_augmented download started this may take some time.
[OK!]
bert_sequence_classifier_ade_augmented download started this may take some time.
[OK!]
setInputCols in MEDICAL_BERT_FOR_SEQUENCE_CLASSIFICATION_3c68e7367d27 expecting 2 columns. Provided column amount: 3. Which should be columns from the following annotators: ['document', 'token']
ner_clinical download started this may take some time.
[OK!]
assertion_jsl_augmented download started this may take some time.
[OK!]
ner_posology download started this may take some time.
[OK!]
ner_deid_generic_augmented download started this may take some time.
[OK!]
assertion_vop_clinical download started this may take some time.
[OK!]
pos_clinical download started this may take some time.
Approximate si

In [13]:

text = """
Ora Hendrickson, a 28-year-old female with a history of gestational diabetes, now type 2 diabetes, and obesity (BMI 33.5 kg/m²), presented with polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior, she completed a five-day course of amoxicillin for a respiratory infection and had been on dapagliflozin for six months.
On examination, she had dry oral mucosa and a benign abdomen. Key lab findings included serum glucose 111 mg/dL, bicarbonate 18 mmol/L, anion gap 20, triglycerides 508 mg/dL, and HbA1c 10%. Venous pH was 7.27, and serum lipase was normal at 43 U/L. Due to poor oral intake, she was admitted for starvation ketosis.
She also reported a two-week headache and anxiety when walking fast. Her father’s paralysis and workplace bullying were significant stressors, leading to insomnia treated with sleeping pills.
Ora, with insulin-dependent type 2 diabetes, coronary artery disease, and chronic renal insufficiency, was previously admitted for acute paraplegia. She developed pressure wounds on her left foot and sacral area. Transferred for further care, she was on multiple medications, including Fragmin, Xenaderm, Lantus, OxyContin, Avandia, and Neurontin. Pathology revealed tumor cells positive for estrogen and progesterone receptors.
Discharged with Avandia, Coumadin, metformin, and Lisinopril, she was also prescribed aspirin and an Albuterol inhaler for asthma.
"""

light_model = LightPipeline(custom_model)
result = light_model.fullAnnotate(text)

In [46]:
result[0]["chunk_merge"]

[Annotation(category, 1, 194, ADE, {'Some(noADE)': '5.2367595E-6', 'entity': 'UNK', 'Some(ADE)': '0.99999475', 'chunk': '0', 'sentence': '0'}, []),
 Annotation(category, 196, 332, ADE, {'Some(noADE)': '7.5360344E-6', 'entity': 'UNK', 'Some(ADE)': '0.9999925', 'chunk': '1', 'sentence': '1'}, []),
 Annotation(category, 334, 394, ADE, {'Some(noADE)': '5.481608E-6', 'entity': 'UNK', 'Some(ADE)': '0.9999945', 'chunk': '2', 'sentence': '2'}, []),
 Annotation(category, 396, 522, ADE, {'Some(noADE)': '6.456064E-6', 'entity': 'UNK', 'Some(ADE)': '0.99999356', 'chunk': '3', 'sentence': '3'}, []),
 Annotation(category, 524, 581, ADE, {'Some(noADE)': '3.3689965E-5', 'entity': 'UNK', 'Some(ADE)': '0.9999663', 'chunk': '4', 'sentence': '4'}, []),
 Annotation(category, 583, 647, ADE, {'Some(noADE)': '5.349816E-6', 'entity': 'UNK', 'Some(ADE)': '0.99999464', 'chunk': '5', 'sentence': '5'}, []),
 Annotation(category, 649, 839, ADE, {'Some(noADE)': '5.2723835E-6', 'entity': 'UNK', 'Some(ADE)': '0.999994

In [28]:
result[0]["summary"]

[Annotation(document, 0, 955, A 28-year-old female with a history of gestational diabetes, type 2 diabetes, and obesity presented with polyuria, polydipsia, poor appetite, and vomiting. She had a five-day course of amoxicillin for a respiratory infection and had been on dapagliflozin for six months. On examination, she had dry oral mucosa and a benign abdomen. Lab findings showed serum glucose 111 mg/dL, bicarbonate 18 mmol/L, anion gap 20, triglycerides 508 mg/dL, and HbA1c 10%. She was admitted for starvation ketosis and a two-week headache and anxiety when walking fast. Her father's paralysis and workplace bullying were significant stressors leading to insomnia treated with sleeping pills. She was on multiple medications including Fragmin, Xenaderm, Lantus, OxyContin, Avandia, and Neurontin. Pathology revealed tumor cells positive for estrogen and progesterone receptors. She was discharged with multiple medications including aspirin and an Albuterol inhaler for asthma., {'score': '0

In [45]:
result[0]["obfuscation"]

,0
0,"Annotation(document, 0, 1400, Humberto Magnus, a 23-year-old female with a history of gestational diabetes, now type 2 diabetes, and obesity (BMI 33.5 kg/m²), presented with polyuria, polydipsia,..."


In [47]:
result[0]["deid_ner_chunk"]

[Annotation(chunk, 1, 15, Ora Hendrickson, {'chunk': '0', 'confidence': '0.95475', 'ner_source': 'deid_ner_chunk', 'entity': 'NAME', 'sentence': '0'}, []),
 Annotation(chunk, 20, 30, 28-year-old, {'chunk': '1', 'confidence': '0.9995', 'ner_source': 'deid_ner_chunk', 'entity': 'AGE', 'sentence': '0'}, [])]

In [48]:

from sparknlp_jsl.pipeline_tracer import PipelineTracer

pipeline_tracer = PipelineTracer(custom_model)

column_maps = pipeline_tracer.createParserDictionary()
column_maps.update({"document_identifier": "explain_clinical_doc_oncology"})
column_maps

{'document_identifier': 'explain_clinical_doc_oncology',
 'document_text': 'document',
 'entities': ['deid_ner_chunk',
  'posology_ner_chunk',
  'clinical_ner_chunk',
  'chunk_merge'],
 'assertions': ['assertion', 'assertion_vop'],
 'resolutions': [{'vocab': 'icd10cm_code',
   'resolver_column_name': 'icd10cm_code'},
  {'vocab': 'rxnorm_code', 'resolver_column_name': 'rxnorm_code'}],
 'relations': ['generic_re'],
 'summaries': ['summary'],
 'deidentifications': [{'original': 'sentence',
   'obfuscated': 'obfuscation',
   'masked': ''}],
 'classifications': [{'classification_column_name': 'ade_classification1',
   'sentence_column_name': 'sentence'},
  {'classification_column_name': 'ade_classification1',
   'sentence_column_name': 'sentence'}]}

In [54]:
from sparknlp_jsl.pipeline_output_parser import PipelineOutputParser

column_maps = {
    'document_identifier': 'some document identifier',
    'document_text': 'document',
    'entities': ['clinical_ner_chunk','posology_ner_chunk','deid_ner_chunk',],
    'assertions': ['assertion_vop', 'assertion'],
    'resolutions': [{
            'vocab':"rxnorm",
            'resolver_column_name': 'rxnorm_code'
        },
        {
            'vocab':"icd10",
            'resolver_column_name': 'icd10cm_code'
    }],
    'relations': ['generic_re'],
    'summaries': ['summary'],
    'deidentifications' : [{
        "original": "document",
        "obfuscated": "obfuscation",
        "masked": None # None, will check in metadata to masked field
    }],
    'classifications':[{
        "classification_column_name": "ade_classification1",
        "sentence_column_name": "sentence",
    }]
}


pipeline_parser = PipelineOutputParser(column_maps,)
result = pipeline_parser.run(result, return_relation_entities=True )

result['result'][0]

{'document_identifier': 'some document identifier',
 'document_id': 0,
 'document_text': ['\nOra Hendrickson, a 28-year-old female with a history of gestational diabetes, now type 2 diabetes, and obesity (BMI 33.5 kg/m²), presented with polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior, she completed a five-day course of amoxicillin for a respiratory infection and had been on dapagliflozin for six months.\nOn examination, she had dry oral mucosa and a benign abdomen. Key lab findings included serum glucose 111 mg/dL, bicarbonate 18 mmol/L, anion gap 20, triglycerides 508 mg/dL, and HbA1c 10%. Venous pH was 7.27, and serum lipase was normal at 43 U/L. Due to poor oral intake, she was admitted for starvation ketosis.\nShe also reported a two-week headache and anxiety when walking fast. Her father’s paralysis and workplace bullying were significant stressors, leading to insomnia treated with sleeping pills.\nOra, with insulin-dependent type 2 diabetes, coronary artery dis

In [58]:
pd.DataFrame(result["result"][0]["entities"])

,chunk_id,chunk,begin,end,ner_label,ner_source,ner_confidence
0,8212ece0,Ora Hendrickson,1,15,NAME,deid_ner_chunk,0.95475
1,cac49f82,28-year-old,20,30,AGE,deid_ner_chunk,0.9995
2,d35ff417,gestational diabetes,57,76,PROBLEM,clinical_ner_chunk,0.95325
3,ecbc13bf,type 2 diabetes,83,97,PROBLEM,clinical_ner_chunk,0.79780006
4,f769b24d,obesity,104,110,PROBLEM,clinical_ner_chunk,0.9972
...,...,...,...,...,...,...,...
67,7f01c6c2,aspirin,1356,1362,TREATMENT,clinical_ner_chunk,0.9986
68,8dbef5e4,aspirin,1356,1362,DRUG,posology_ner_chunk,0.9968
69,0a3d1173,an Albuterol inhaler,1368,1387,TREATMENT,clinical_ner_chunk,0.9763667
70,88ee6291,Albuterol,1371,1379,DRUG,posology_ner_chunk,0.9738


In [59]:
pd.DataFrame(result["result"][0]["assertions"])

,chunk_id,chunk,assertion,assertion_confidence,assertion_source
0,d35ff417,gestational diabetes,Past,0.6894,assertion
1,ecbc13bf,type 2 diabetes,Family,1.0,assertion
2,f769b24d,obesity,Present,0.9754,assertion
3,baf4ee42,BMI,Past,0.9952,assertion
4,4ba206e3,polyuria,Present,0.991,assertion
5,090833c0,polydipsia,Family,0.5674,assertion
6,12396d0a,poor appetite,Present,0.8387,assertion
7,1caa3436,vomiting,Family,0.9999,assertion
8,7a646465,amoxicillin,Past,1.0,assertion
9,0c13e631,a respiratory infection,Present,1.0,assertion


In [60]:
pd.DataFrame(result["result"][0]["deidentifications"])

,original,obfuscated,masked
0,"[\nOra Hendrickson, a 28-year-old female with a history of gestational diabetes, now type 2 diabetes, and obesity (BMI 33.5 kg/m²), presented with polyuria, polydipsia, poor appetite, and vomiting...","[ Humberto Magnus, a 23-year-old female with a history of gestational diabetes, now type 2 diabetes, and obesity (BMI 33.5 kg/m²), presented with polyuria, polydipsia, poor appetite, and vomiting....","[ <NAME>, a <AGE> female with a history of gestational diabetes, now type 2 diabetes, and obesity (BMI 33.5 kg/m²), presented with polyuria, polydipsia, poor appetite, and vomiting. Two weeks prio..."


In [63]:
pd.DataFrame(result["result"][0]["relations"])

,relation,chunk1_id,chunk1,entity1,entity1_begin,entity1_end,chunk2_id,chunk2,entity2,entity2_begin,entity2_end,confidence,direction
0,DRUG-DRUG,7252a3a2,Fragmin,DRUG,1127,1133,32aee897,Xenaderm,DRUG,1136,1143,1.0,both
1,DRUG-DRUG,7252a3a2,Fragmin,DRUG,1127,1133,e419b1dd,Lantus,DRUG,1146,1151,1.0,both
2,DRUG-DRUG,7252a3a2,Fragmin,DRUG,1127,1133,5b3b5ece,OxyContin,DRUG,1154,1162,1.0,both
3,DRUG-DRUG,7252a3a2,Fragmin,DRUG,1127,1133,094a24c9,Avandia,DRUG,1165,1171,1.0,both
4,DRUG-DRUG,7252a3a2,Fragmin,DRUG,1127,1133,3bc05671,Neurontin,DRUG,1178,1186,1.0,both
5,DRUG-DRUG,32aee897,Xenaderm,DRUG,1136,1143,e419b1dd,Lantus,DRUG,1146,1151,1.0,both
6,DRUG-DRUG,32aee897,Xenaderm,DRUG,1136,1143,5b3b5ece,OxyContin,DRUG,1154,1162,1.0,both
7,DRUG-DRUG,32aee897,Xenaderm,DRUG,1136,1143,094a24c9,Avandia,DRUG,1165,1171,1.0,both
8,DRUG-DRUG,32aee897,Xenaderm,DRUG,1136,1143,3bc05671,Neurontin,DRUG,1178,1186,1.0,both
9,DRUG-DRUG,e419b1dd,Lantus,DRUG,1146,1151,5b3b5ece,OxyContin,DRUG,1154,1162,1.0,both


In [64]:
pd.DataFrame.from_dict(result["result"][0]["classifications"])

,category,sentence,sentence_id
0,ADE,"Ora Hendrickson, a 28-year-old female with a history of gestational diabetes, now type 2 diabetes, and obesity (BMI 33.5 kg/m²), presented with polyuria, polydipsia, poor appetite, and vomiting.",0
1,ADE,"Two weeks prior, she completed a five-day course of amoxicillin for a respiratory infection and had been on dapagliflozin for six months.",1
2,ADE,"On examination, she had dry oral mucosa and a benign abdomen.",2
3,ADE,"Key lab findings included serum glucose 111 mg/dL, bicarbonate 18 mmol/L, anion gap 20, triglycerides 508 mg/dL, and HbA1c 10%.",3
4,ADE,"Venous pH was 7.27, and serum lipase was normal at 43 U/L.",4
5,ADE,"Due to poor oral intake, she was admitted for starvation ketosis.",5
6,ADE,"She also reported a two-week headache and anxiety when walking fast. Her father’s paralysis and workplace bullying were significant stressors, leading to insomnia treated with sleeping pills.",6
7,ADE,"Ora, with insulin-dependent type 2 diabetes, coronary artery disease, and chronic renal insufficiency, was previously admitted for acute paraplegia.",7
8,ADE,She developed pressure wounds on her left foot and sacral area.,8
9,ADE,"Transferred for further care, she was on multiple medications, including Fragmin, Xenaderm, Lantus, OxyContin, Avandia, and Neurontin.",9
